In [10]:
#auto reload 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import pandas as pd
import numpy as np
import os 
from datasets import load_dataset, load_from_disk
import torch 

In [12]:
import pandas as pd 
 
bow_english = pd.read_csv("data/unigram_freq_en.csv")
bow_french = pd.read_csv("data/unigram_freq_fr.csv")



In [13]:
n, m = 30000, 30000

bow_english = bow_english[:n]
bow_french = bow_french[:m]

In [14]:
train_len, val_len = 10000, 1000
#train_len, val_len = None, None

In [15]:
wmt14 = load_dataset("wmt14", "fr-en", data_dir="data/")

# Accessing example data
train_data = wmt14["train"]
val_data = wmt14["validation"]

if train_len is not None:
    train_data = train_data.select(range(train_len))
else :
    train_len = len(train_data)
if val_len is not None:
    val_data = val_data.select(range(val_len))
else : 
    val_len = len(val_data) 


/Users/nadia/anaconda3/envs/projetmla/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [16]:
examples = 4 
#check some data and their translation
for i, t in zip(range(examples), train_data):
    ex = t["translation"]
    print(ex["fr"], " -> ", ex["en"], "\n\n")

Reprise de la session  ->  Resumption of the session 


Je déclare reprise la session du Parlement européen qui avait été interrompue le vendredi 17 décembre dernier et je vous renouvelle tous mes vux en espérant que vous avez passé de bonnes vacances.  ->  I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period. 


Comme vous avez pu le constater, le grand "bogue de l'an 2000" ne s'est pas produit. En revanche, les citoyens d'un certain nombre de nos pays ont été victimes de catastrophes naturelles qui ont vraiment été terribles.  ->  Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful. 


Vous avez souhaité un débat à ce sujet dans les prochains jours, au cours de cette période de session.  ->  You 

In [17]:
from sacremoses import MosesTokenizer, MosesDetokenizer

# Sample sentence
sentence = "This is an  example sentence with words that may not be in the shortlist."

mt_en = MosesTokenizer(lang="en")
mt_fr = MosesTokenizer(lang="fr")

tokenized = mt_en.tokenize(sentence)
tokenized


['This',
 'is',
 'an',
 'example',
 'sentence',
 'with',
 'words',
 'that',
 'may',
 'not',
 'be',
 'in',
 'the',
 'shortlist',
 '.']

In [18]:
class TokenizerWrapper:
    def __init__(self, tokenizer_en, tokenizer_fr):
        self.tokenizer_en = tokenizer_en
        self.tokenizer_fr = tokenizer_fr
    
    def preprocess_text(self, text):
        # Add any additional preprocessing steps here
        # For example, you can convert text to lowercase, remove special characters, etc.
        # Modify this function based on your specific requirements.
        return text.lower()

    def tokenize_function(self, examples):
        preprocessed_en = self.preprocess_text(examples["translation"]["en"])
        preprocessed_fr = self.preprocess_text(examples["translation"]["fr"])

        return {"tokenized_en": self.tokenizer_en.tokenize(preprocessed_en),
                "tokenized_fr": self.tokenizer_fr.tokenize(preprocessed_fr)}

tokenizer_wrapper = TokenizerWrapper(mt_en, mt_fr)


In [19]:
if not os.path.exists("processed_data/tokenized_train_data{}".format(train_len)):
    tokenized_train_data  = train_data.map(tokenizer_wrapper.tokenize_function, batched=False, num_proc=19, remove_columns=["translation"])
    tokenized_train_data.save_to_disk("processed_data/tokenized_train_data{}".format(train_len))


Saving the dataset (1/1 shards): 100%|██████████| 10000/10000 [00:00<00:00, 57883.13 examples/s]


In [20]:
    
if not os.path.exists("processed_data/tokenized_val_data{}".format(val_len)):
    tokenized_val_data  = val_data.map(tokenizer_wrapper.tokenize_function, batched=False, num_proc=19, remove_columns=["translation"])
    tokenized_val_data.save_to_disk("processed_data/tokenized_val_data{}".format(val_len))


Saving the dataset (1/1 shards): 100%|██████████| 1000/1000 [00:00<00:00, 135125.77 examples/s]


In [21]:
tokenized_train_data = load_from_disk("processed_data/tokenized_train_data{}".format(train_len))
tokenized_val_data = load_from_disk("processed_data/tokenized_val_data{}".format(val_len))

In [22]:
most_frequent_english_words = bow_english["word"].apply(lambda x: str(x)).tolist()
most_frequent_french_words = bow_french["word"].apply(lambda x: str(x)).tolist()
tokenized_most_frequent_english_words = mt_en.tokenize(" ".join(most_frequent_english_words))[:n]
tokenized_most_frequent_french_words = mt_fr.tokenize(" ".join(most_frequent_french_words))[:m]

In [23]:
len(tokenized_most_frequent_french_words), len(tokenized_most_frequent_english_words)

(30000, 30000)

In [24]:
class toIdTransform:
    def __init__(self, most_frequent_words_en, most_frequent_words_fr, tor):
        self.most_frequent_words_en = most_frequent_words_en
        self.most_frequent_words_fr = most_frequent_words_fr
        self.word_to_id_en = {word: i for i, word in enumerate(most_frequent_words_en)}
        self.word_to_id_fr = {word: i for i, word in enumerate(most_frequent_words_fr)}
        self.tor = tor
    
    def __call__(self, tokenized):
        return {"ids_en":[self.word_to_id_en.get(token, len(self.most_frequent_words_en)) for token in tokenized["tokenized_en"]],
                "ids_fr":[self.word_to_id_fr.get(token, len(self.most_frequent_words_fr)) for token in tokenized["tokenized_fr"]]}
    
to_id_transform = toIdTransform(tokenized_most_frequent_english_words, tokenized_most_frequent_french_words, torch.tensor)


In [25]:
if not os.path.exists("processed_data/id_train_data{}".format(train_len)):
    tokenized_train_data = tokenized_train_data.map(to_id_transform, batched=False, num_proc=19)
    tokenized_train_data.save_to_disk("processed_data/id_train_data{}".format(train_len))


Saving the dataset (1/1 shards): 100%|██████████| 10000/10000 [00:00<00:00, 501861.08 examples/s]


In [26]:
if not os.path.exists("processed_data/id_val_data{}".format(val_len)):
    tokenized_val_data = tokenized_val_data.map(to_id_transform, batched=False, num_proc=19)
    tokenized_val_data.save_to_disk("processed_data/id_val_data{}".format(val_len))

Saving the dataset (1/1 shards): 100%|██████████| 1000/1000 [00:00<00:00, 59489.45 examples/s]


In [27]:
tokenized_train_data = load_from_disk("processed_data/id_train_data{}".format(train_len))
tokenized_val_data = load_from_disk("processed_data/id_val_data{}".format(val_len))

In [28]:
tokenized_train_data[0]

{'tokenized_en': ['resumption', 'of', 'the', 'session'],
 'tokenized_fr': ['reprise', 'de', 'la', 'session'],
 'ids_en': [30000, 1, 0, 1497],
 'ids_fr': [17227, 22350, 10301, 13488]}

In [29]:
Tx = 30 
Ty = 30


    

In [30]:
def pad_to_length(x, length, pad_value):
    if len(x) < length:
        return x + [pad_value] * (length - len(x))
    else:
        return x[:length]

idx_train_tensor_en = torch.zeros((len(tokenized_train_data), Tx), dtype=torch.int16)
idx_train_tensor_fr = torch.zeros((len(tokenized_train_data), Ty), dtype=torch.int16)

for i, x in enumerate(tokenized_train_data):
    idx_train_tensor_en[i] = torch.tensor(pad_to_length(x["ids_en"], Tx, m+1))
    idx_train_tensor_fr[i] = torch.tensor(pad_to_length(x["ids_fr"], Ty, n+1))

In [32]:
idx_train_tensor_fr[500]

tensor([19328,   473,   109, 28542,  8254, 15510,   721, 30000, 10433,  8141,
        30000,  2301,   109,   708,  4907,   492, 30000,  2635, 30000, 27924,
           34,   109, 30000, 30000, 30000, 30001, 30001, 30001, 30001, 30001],
       dtype=torch.int16)

In [18]:
class to_tensor:
    def __init__(self, tor):
        self.tor = tor
    
    def __call__(self, ids):
        return {"ids_en":self.tor.tensor(ids["ids_en"]),
                "ids_fr":self.tor.tensor(ids["ids_fr"])}
    
tokenized_train_data = tokenized_train_data.map(to_tensor(torch), batched=False, num_proc=19)
tokenized_val_data = tokenized_val_data.map(to_tensor(torch), batched=False, num_proc=19)

Map (num_proc=19): 100%|██████████| 1000/1000 [00:00<00:00, 6781.27 examples/s]


In [26]:
def to_vector(ids, vocab_size):
    vecs = torch.zeros(len(ids), vocab_size)
    vecs[range(len(ids)), ids] = 1
    return vecs

class to_vec:
    def __init__(self, tor, vocab_size, lang):
        self.tor = tor
        self.vocab_size = vocab_size
        self.lang = lang

    
    def __call__(self, ids):
        ids = self.tor.tensor(ids["ids_{}".format(self.lang)])
        vecs = self.tor.zeros(len(ids), self.vocab_size)
        vecs[range(len(ids)), ids] = 1
        return vecs

In [59]:
from models.fcnn import FCNN

model = FCNN(len(tokenized_most_frequent_english_words), [10,10], 30000, "cuda" if torch.cuda.is_available() else "cpu",torch.nn.ReLU())

In [26]:
for i in range(len(tokenized_train_data)):

    tensorized = torch.tensor(tokenized_train_data[i]["ids_en"]).to("cuda" if torch.cuda.is_available() else "cpu")
    
    result = model(to_vector(tensorized, len(tokenized_most_frequent_english_words)).to("cuda" if torch.cuda.is_available() else "cpu"))
    
    print(result.shape)
    
    break

torch.Size([4, 30000])


In [27]:
from models.rnn import RNN

encoder_rnn = RNN(len(tokenized_most_frequent_english_words),100,2,"cuda" if torch.cuda.is_available() else "cpu",torch.nn.Tanh())

In [40]:
for x in tokenized_train_data:
    tensorized = torch.tensor(x["ids_en"]).to("cuda" if torch.cuda.is_available() else "cpu")
    result = encoder_rnn(to_vector(tensorized, len(tokenized_most_frequent_english_words)).to("cuda" if torch.cuda.is_available() else "cpu").view(1,-1,len(tokenized_most_frequent_english_words)))
    print(result[0].shape)
    break
print (encoder_rnn)

torch.Size([1, 4, 100])
RNN(
  (rnn): RNN(30001, 100, num_layers=2, batch_first=True)
)


In [ ]:
from models.rnn import RNN
from models.enco_deco import Encoder, Decoder  

# Création de l'encodeur
encoder = Encoder(len(tokenized_most_frequent_english_words), 256,512, 2, "cuda" if torch.cuda.is_available() else "cpu", 0.5)
print(len(tokenized_most_frequent_english_words))
print(encoder)

for x in tokenized_train_data:
    tensorized = torch.tensor(x["ids_en"]).to("cuda" if torch.cuda.is_available() else "cpu").long()
    print(tensorized)
    result = encoder(to_vector(tensorized, len(tokenized_most_frequent_english_words)).to("cuda" if torch.cuda.is_available() else "cpu").view(1, -1, len(tokenized_most_frequent_english_words)))

    print(result[0].shape)
    break

# Création du décodeur
decoder = Decoder(len(tokenized_most_frequent_english_words), 100, 2, "cuda" if torch.cuda.is_available() else "cpu", torch.nn.Tanh())


decoded_output, hidden_state = decoder(torch.tensor([1]), result[0], None)
print("Decoded Output shape:", decoded_output.shape)
print("Hidden State shape:", hidden_state.shape)

BLEU SCORE

1-EXISTANT NLTK

In [62]:
import nltk 
hypothesis=['Le','chat','se','trouve','sur','le','toit']
reference=['Le','chat','se','trouve','sur','le','toit','bla']
BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis) 
print(BLEUscore)


0.8668778997501817


2-SCORE PERSONNALISE

In [100]:
import collections
import math

def ngram_precision(candidate, reference, n):
    candidate_ngrams = [tuple(candidate[i:i+n]) for i in range(len(candidate)-n+1)]
    reference_ngrams = [tuple(reference[i:i+n]) for i in range(len(reference)-n+1)]

    candidate_ngram_counts = collections.Counter(candidate_ngrams)
    reference_ngram_counts = collections.Counter(reference_ngrams)

    overlap_count = sum((candidate_ngram_counts & reference_ngram_counts).values())
    total_count = max(1, len(candidate_ngrams))

    precision = overlap_count / total_count
    return precision

def bleu_score(candidate, references, weights):
    precisions = []

    for i in range(len(weights)):
        n = i + 1
        precision = ngram_precision(candidate, references, n)
        precisions.append(precision)

    geometric_mean = math.exp(sum(weights[i] * math.log(p) for i, p in enumerate(precisions)))

    brevity_penalty = min(1, len(candidate) / max(len(reference) for reference in references))

    bleu = brevity_penalty * geometric_mean
    return bleu

# Exemple d'utilisation
reference = ['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog']
candidate = ['the', 'fast', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog']

candidate=['Le','chat','se','trou','s','le','to','bli']
reference=['Le','chat','se','trouve','sur','le','toit']

weights = [0.25,0.25,0.25] # You can adjust the weights based on your preference

score = bleu_score(candidate, reference, weights)
print("BLEU Score:", score)


BLEU Score: 0.392814650900513


In [3]:
import torch 

#matmul tests: 
batch_size = 2 
Tx = 3 
hidden_size = 4
a = torch.randint(0,10,(batch_size, Tx),dtype=torch.float)
h = torch.randint(0,10,(batch_size, Tx, hidden_size),dtype=torch.float)


In [4]:
a

tensor([[9., 4., 7.],
        [0., 1., 6.]])

In [18]:
h.swapaxes(1,2)

tensor([[[0., 0., 3.],
         [7., 1., 2.],
         [9., 5., 0.],
         [5., 6., 9.]],

        [[8., 0., 7.],
         [0., 0., 7.],
         [9., 1., 0.],
         [2., 3., 1.]]])

In [16]:
h.shape,a.shape

(torch.Size([2, 3, 4]), torch.Size([2, 3]))

In [17]:
(h.swapaxes(1,2) @ a.unsqueeze(2)).squeeze()

tensor([[ 21.,  81., 101., 132.],
        [ 42.,  42.,   1.,   9.]])